In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pydicom
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import preprocess_input
from keras.utils import to_categorical

In [2]:
!pip install -U pydicom
!pip install -U pillow
!pip install -U numpy
!pip install -U opencv-python-headless
!pip install -U python-gdcm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 20.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.5.4.60
    Uninstalling opencv-python-headless-4.5.4.60:
      Successfully uninstalled opencv-python-headless-4.5.4.60
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
easyocr 1.6.2 requires opencv-python-headless<=4.5.4.60, but you have opencv-python-headless 4.7.0.72 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 60.0 MB/s eta 0:00:00


In [3]:
data_dir = '/kaggle/input/rsna-2022-cervical-spine-fracture-detection/train_images'
csv_path = '/kaggle/input/rsna-2022-cervical-spine-fracture-detection/train.csv'
images_dir = '/kaggle/working/images_dir'
labels = []

In [4]:
import shutil

shutil.rmtree('/kaggle/working/images')
shutil.rmtree('/kaggle/working/labels')
#shutil.rmtree('/kaggle/working/training')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/images'

In [ ]:
shutil.rmtree('/kaggle/working/training')

In [ ]:

os.makedirs('images', exist_ok=True)
os.makedirs('labels', exist_ok=True)

In [ ]:

train_df = pd.read_csv(csv_path)
labels=[]
images = np.empty((0, 224, 224))

In [ ]:
# import gdcm, pylibjpeg
images = np.empty((0, 224, 224))
os.makedirs('training', exist_ok=True)

counter=0

for subdir in os.listdir(data_dir):
    if(counter < 20):
        counter+=1
        print(counter)
       
        if os.path.isdir(os.path.join(data_dir, subdir)):
            print('running')
            
            study_uid = subdir
          
            label = train_df[train_df['StudyInstanceUID'] == study_uid]['patient_overall'].values[0]
            
            for file in os.listdir(os.path.join(data_dir, subdir)):
                
                if file.endswith('.dcm'):
                    
                    try:
                        dcm_file = pydicom.read_file(os.path.join(data_dir, subdir, file))
                        dcm_file.PhotometricInterpretation = 'YBR_FULL'
                        
                        img = dcm_file.pixel_array.astype(float)
                        img = (img / np.max(img)) * 255.0
                   
                        img = Image.fromarray(img.astype('uint8'))
                        img = img.resize((224,224))
                        img.save(os.path.join('training', str(file) + '.jpg'))
                        img_arr = np.array(img)
                        images = np.concatenate((images, [img_arr]))
                        labels.append(label)
                    
                    except:
                        print("Skipping file:", file)
                        continue
                    
                

In [ ]:

images=np.array(images)
labels=np.array(labels)

In [ ]:
labels.shape

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, random_state = 42)

In [ ]:
print(f'x_train: {len(x_train)}')
print(f'y_train: {len(y_train)}')
print(f'x_test: {len(x_test)}')
print(f'y_test: {len(y_test)}')

In [ ]:
x_test.shape

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], -1))
x_test = np.reshape(x_test, (x_test.shape[0], -1))

In [ ]:
x_test.shape

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import time

knn = KNeighborsClassifier(n_neighbors=5)
start_time = time.time()
knn_history = knn.fit(x_train, y_train)
y_pred_knn = knn.predict(x_test) 
acc_knn = accuracy_score(y_test, y_pred_knn)
end_time = time.time()

print(acc_knn)
print("Total training and testing time:", end_time - start_time, "seconds")

In [ ]:

import time

y_pred_knn = knn.predict(x_test)  
print(y_pred_knn)

In [ ]:
plt.plot(3, knn_history.history['accuracy'], 'g', label='accuracy')
plt.plot(3, knn_history.history['loss'], 'b', label='loss')
plt.title('Accuracy vs. Loss for KNN')
plt.xlabel('Epochs')
plt.ylabel('Percentage')
plt.legend()
plt.show()

In [ ]:
x_test.shape


In [ ]:
acc = accuracy_score(y_test, y_pred_knn)
print(acc)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)

start_time = time.time()
dt_history = dt.fit(x_train, y_train)
y_pred_dt = dt.predict(x_test) 
acc_dt = accuracy_score(y_test, y_pred_dt)
end_time = time.time()
print(acc_dt)
print("Total training and testing time:", end_time - start_time, "seconds")

In [ ]:
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
from sklearn.ensemble import VotingClassifier
voting_classifier_soft = VotingClassifier(
    estimators = [('dt',dt),
                  ('knn', knn)], voting='soft')


In [ ]:
import time

start_time = time.time()

voting_classifier_soft.fit(x_train, y_train)
y_pred_soft = voting_classifier_soft.predict(x_test)
acc_soft = accuracy_score(y_test, y_pred_soft)
end_time = time.time()
print(acc_soft)

In [ ]:
print(end_time-start_time)